In [56]:
import json
import pandas as pd
import ast
from pandas import json_normalize
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

In [57]:
rows=[]

with open("datasets/australian_user_reviews.json",encoding="utf-8")as file: 
    for line in file.readlines():
        rows.append(ast.literal_eval(line))

reviews = pd.DataFrame(rows) 

In [58]:
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [59]:
reviews = pd.concat([pd.concat([pd.DataFrame(row['reviews']), pd.Series([row['user_id']] * len(row['reviews']), name='user_id')], axis=1) for _, row in reviews.iterrows()], ignore_index=True)
reviews.shape

(59305, 8)

In [60]:
reviews.drop_duplicates(inplace=True)
reviews.shape

(58431, 8)

In [61]:
reviews['posted'].tail()

59300    Posted July 10.
59301     Posted July 8.
59302     Posted July 3.
59303    Posted July 20.
59304     Posted July 2.
Name: posted, dtype: object

In [62]:
reviews = reviews.drop(columns=['funny', 'last_edited', 'helpful'])

In [63]:
reviews["posted"]

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59300              Posted July 10.
59301               Posted July 8.
59302               Posted July 3.
59303              Posted July 20.
59304               Posted July 2.
Name: posted, Length: 58431, dtype: object

In [64]:
# Eliminar la palabra "Posted" de la columna 'posted'
reviews['posted'] = reviews['posted'].str.replace('Posted ', '')

# Convertir la columna 'posted' a formato de fecha
reviews['posted'] = pd.to_datetime(reviews['posted'], errors='coerce')

# Extraer solo el año de la fecha y lo usamos en una nueva columna llamada "posted_year"
reviews['posted_year'] = reviews['posted'].dt.year

# Borramos la columna "posted"
reviews = reviews.drop('posted', axis=1)

In [65]:
reviews = reviews.dropna(subset=['posted_year'])
reviews['posted_year'] = reviews['posted_year'].astype(int)

In [66]:
reviews

,item_id,recommend,review,user_id,posted_year
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011
3,251610,True,I know what you think when you see this title ...,js41637,2014
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013
...,...,...,...,...,...
59252,730,True,its FUNNNNNNNN,wayfeng,2015
59255,253980,True,Awesome fantasy game if you don't mind the gra...,76561198251004808,2015
59265,730,True,Prettyy Mad Game,72947282842,2015
59267,730,True,AMAZING GAME 10/10,ApxLGhost,2015


In [67]:
reviews['review'].fillna('', inplace=True)

C:\Users\juan_\AppData\Local\Temp\ipykernel_16668\240426863.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['review'].fillna('', inplace=True)


In [68]:
def analisis_sentimiento(sentimiento):

    # Sentimiento positivo
    if sentimiento >= 0.05:
        return 2 
    # Sentimiento negativo
    elif sentimiento <= -0.04:
        return 0
    # Sentimiento neutral
    else:
        return 1

In [69]:
# Inicializar el analizador de sentimientos de NLTK
sid = SentimentIntensityAnalyzer()

In [70]:
reviews['sentiment_score'] = reviews['review'].apply(lambda x: sid.polarity_scores(x)['compound'])
reviews['sentiment_analysis'] = reviews['sentiment_score'].apply(analisis_sentimiento)

In [71]:
reviews.drop(columns=['review', 'sentiment_score'],inplace=True)

In [72]:
reviews.reset_index(drop=True, inplace=True)
reviews

,item_id,recommend,user_id,posted_year,sentiment_analysis
0,1250,True,76561197970982479,2011,2
1,22200,True,76561197970982479,2011,2
2,43110,True,76561197970982479,2011,2
3,251610,True,js41637,2014,2
4,227300,True,js41637,2013,2
...,...,...,...,...,...
48493,730,True,wayfeng,2015,1
48494,253980,True,76561198251004808,2015,2
48495,730,True,72947282842,2015,0
48496,730,True,ApxLGhost,2015,2


In [74]:
reviews.to_parquet("datasets/Parquet/reviews.parquet", index=False)